# Youtube Rank

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [4]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [6]:
channel = channel_list[0]
rank = channel.select_one('.rank').get_text().strip()
rank = int(rank)
rank

1

In [7]:
category = channel.select_one('p.category').get_text().strip()
category[1:-1]

'음악/댄스/가수'

In [8]:
name = channel.select_one('.subject a').get_text().strip()
name

'BLACKPINK'

In [9]:
subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
view = channel.select_one('.view_cnt').get_text().strip()
video = channel.select_one('.video_cnt').get_text().strip()
subscriber, view, video

('7020만', '214억5709만', '395개')

- 반복문으로 페이지 내에 있는 데이터 수집하기

In [11]:
lines = []
for channel in channel_list:
    rank = int(channel.select_one('.rank').get_text().strip())
    category = channel.select_one('p.category').get_text().strip()
    name = channel.select_one('.subject a').get_text().strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
    view = channel.select_one('.view_cnt').get_text().strip()
    video = channel.select_one('.video_cnt').get_text().strip()
    lines.append([rank,category[1:-1],name,subscriber,view,video])
df = pd.DataFrame(lines, columns=['순위','카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,7020만,214억5709만,395개
1,2,음악/댄스/가수,HYBE LABELS,6300만,204억9684만,725개
2,3,음악/댄스/가수,BANGTANTV,6150만,140억7794만,"1,646개"
3,4,음악/댄스/가수,SMTOWN,2930만,230억1641만,"3,779개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개


- 숫자로 바꿔주는 함수

In [12]:
def convert_unit(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return f'{int(s):,d}'

In [140]:
convert_unit('123억5200만')

'12,352,000,000'

In [13]:
df['구독자수'] = df.구독자수.apply(convert_unit)
df['조회수'] = df.조회수.apply(convert_unit)
df['비디오수'] = df.비디오수.apply(convert_unit)
df.head()

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"70,200,000","21,457,090,000",395
1,2,음악/댄스/가수,HYBE LABELS,"63,000,000","20,496,840,000",725
2,3,음악/댄스/가수,BANGTANTV,"61,500,000","14,077,940,000","1,646"
3,4,음악/댄스/가수,SMTOWN,"29,300,000","23,016,410,000","3,779"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


- 두번째 페이지

In [14]:
# Xpath로 찾아가기 - //*[@id="list-skin"]/nav/span/a[1]
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()
time.sleep(2)

In [143]:
#driver.close()

In [15]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [153]:
lines = []
for channel in channel_list:
    rank = int(channel.select_one('.rank').get_text().strip())
    category = channel.select_one('p.category').get_text().strip()
    name = channel.select_one('.subject a').get_text().strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
    view = channel.select_one('.view_cnt').get_text().strip()
    video = channel.select_one('.video_cnt').get_text().strip()
    lines.append([rank,category[1:-1],name,convert_unit(subscriber),
                  convert_unit(view),convert_unit(video)])

df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널명', '구독자수', '조회수', '비디오수'])
df()

,순위,카테고리,채널명,구독자수,조회수,비디오수
95,96,키즈/어린이,로미유 스토리[Romiyu Story],"4,170,000","1,095,240,000",434
96,97,음식/요리/레시피,[Dorothy]도로시,"4,160,000","998,000,000",905
97,98,취미/라이프,JaeYeol ASMR 재열,"3,980,000","1,096,060,000","1,018"
98,99,키즈/어린이,CoCosToy 꼬꼬스토이,"3,880,000","2,187,780,000",652
99,100,BJ/인물/연예인,허팝Heopop,"3,840,000","3,260,020,000","1,900"


In [16]:
lines = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        rank = int(channel.select_one('.rank').get_text().strip())
        category = channel.select_one('p.category').get_text().strip()
        name = channel.select_one('.subject a').get_text().strip()
        subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
        view = channel.select_one('.view_cnt').get_text().strip()
        video = channel.select_one('.video_cnt').get_text().strip()
        lines.append([rank,category[1:-1],name,convert_unit(subscriber),
                    convert_unit(view),convert_unit(video)])

df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널명', '구독자수', '조회수', '비디오수'])
df.tail()       

,순위,카테고리,채널명,구독자수,조회수,비디오수
995,996,음악/댄스/가수,Reynah,"550,000","148,880,000",246
996,997,TV/방송,J.Y. Park,"550,000","93,730,000",90
997,998,게임,[ALTUBE] 김성현TV,"550,000","422,100,000","2,121"
998,1000,게임,방앗간 비둘기,"550,000","356,900,000","4,381"
999,999,게임,빅헤드,"550,000","326,600,000","3,540"


In [17]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오수
995,996,음악/댄스/가수,Reynah,"550,000","148,880,000",246
996,997,TV/방송,J.Y. Park,"550,000","93,730,000",90
997,998,게임,[ALTUBE] 김성현TV,"550,000","422,100,000","2,121"
998,1000,게임,방앗간 비둘기,"550,000","356,900,000","4,381"
999,999,게임,빅헤드,"550,000","326,600,000","3,540"


In [156]:
df.to_csv('유튜브랭킹.csv', index=False)

In [157]:
driver.close()